# HPE SimpliVity - Prometheus - Grafana Monitoring

## Connect to the Kubernetes environment

In [1]:
kubectl get nodes
kubectl -n svtprometheus get pods,svc

NAME           STATUS   ROLES                  AGE     VERSION
suo04k8sr-c1   Ready    control-plane,worker   3h55m   v1.32.2
suo04k8sr-c2   Ready    control-plane,worker   3h19m   v1.32.2
suo04k8sr-c3   Ready    control-plane,worker   171m    v1.32.2
suo04k8sr-w1   Ready    worker                 169m    v1.32.2
suo04k8sr-w2   Ready    worker                 166m    v1.32.2
suo04k8sr-w3   Ready    worker                 165m    v1.32.2
No resources found in svtprometheus namespace.


## 1. Create the namespace

Create a namespace for your deployment. The example below creates the namespace svtprometheus. 

In [2]:
kubectl apply -f namespace.yml

namespace/prometheus created


In [1]:
# Example content of the namespace.yml file - either run the command above or this one to create the namespace
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Namespace
metadata:
  name: svtprometheus
EOF

namespace/svtprometheus created


## 2. Create the ConfigMap

The connection and monitoring parameters are transferred to the HPE SimpliVity Prometheus connector as a Kubernetes ConfigMap. 

The ConfigMap can be created using th Python script: CreateConfigMap.py. 

Requirements:
    - System with Python 3 and the following Python packages installed: 
      - Fernet
      - getpass
      - etree

Run the script with the following command: python3 CreateConfigMap.py
The script will ask for the following information:
    - username               vCenter username 
    - password               vCenter password
    - OVC/MVA IP address     IP address that the connector uses to connect to the federation
    - name                   name of the yml-file (<name>.yml) and the configmap: <name>-xml
    - port                   TCP Port that the connector uses to publish the counters. 

It will create the Kubernetes yml-file (<name>.yml) that can be used to  create the Configmap <name>-xml 

The CreateConfigMap.py scripts sets for some of the SimpliVity connector parameters default values that can be edited in the file if needed:
    
    Parameter               Default value   Comment
    - namespace             svtprometheus   adjust this to your namespace)
    - timerange             30
    - resolution            SECOND
    - monitoringinterval    30
    - monitor               fcn
    - cluster               
    - limit                 500 
    - offset                -1  
    

In [7]:
pwd
ls

/data/jupyter/deployments/Prometheus/SimpliVity-Prometheus-4.1
CreateConfigMap.py               svtdemo.yml
namespace.yml                    svtdemo.yml.old
SimpliVityClass.py               svtinfrastructure.key
SimpliVity-Prometheus-4.1.ipynb  svtinfrastructure.xml
svtconnect.k8s.Dockerfile        svtinfrastructure.yml
svtconnector.yml                 svtinfra.yml
svtdemo.key                      svtPromConnector.v4.1.py
svtdemo.xml                      SvtPromDeployment.ipynb


In [23]:
# apply the configmap (in the following example the name svtdemo was used)
kubectl apply -f svtdemo.yml

configmap/svtdemo-xml created


In [25]:
kubectl apply -f svtinfra.yml

configmap/svtinfra-xml created


In [ ]:
# Example for the SimpliVity Prometheus Connector Configmap = do not use this example code.
# Create your own configmap using the CreateConfigMap.py script! 
apiVersion: v1
kind: ConfigMap
metadata:
  name: svtdemo-xml
  namespace: test
data:
  svtconnector.key: |- 
    SgR7_7aCa0YegMcLx0ivwDNA_qtZNil2aeZCpWb5vws=
  svtconnector.xml: |- 
    <data> 
      <username>SvtCollector@vsphere.local</username>
      <user>gAAAAABfWhQVvn8PxLT86MLghsqaeB56fpGK2GdyyZTnkuym18udWAl_i51Ddo51-ScwwVUSST2uIqHq7vtyyWqOZ5MVtvv2ESVMZ88tTiJijzoxAHNA_2Y=</user>
      <password>gAAAAABfWhQV0DDfLv-prmLnU1ZMMEwXwJxdUugCHxcIjqOQDt7ePaN1d5ZAR96ox-qHJSaHF87B74hek5DvWleMRq__77Fogg==</password>
      <ovc>10.1.40.15</ovc>
      <timerange>30</timerange>
      <resolution>SECOND</resolution>
      <monitoringintervall>30</monitoringintervall>
      <logfile>svtdemo-xml.log</logfile>
      <port>9091</port>
      <monitor>fcn</monitor>
      <cluster></cluster>
      <limit>500</limit>
      <offset>-1</offset>
    </data>
EOF  

Check that the configmap is created properly

In [29]:
kubectl -n prometheus get configmap

NAME               DATA   AGE
kube-root-ca.crt   1      143m
svtdemo-xml        2      12m
svtinfra-xml       2      2m13s


In [22]:
kubectl -n prometheus delete configmap/svtdemo-xml

configmap "svtdemo-xml" deleted


In [24]:
kubectl -n prometheus describe configmap/svtdemo-xml

Name:         svtdemo-xml
Namespace:    prometheus
Labels:       <none>
Annotations:  <none>

Data
====
svtconnector.xml:
----
<data> 
  <username>SvtCollector@vsphere.local</username>
  <user>gAAAAABn6p9M4nlNVN27GKiijxeKDXmn_EWpyCvay2xH-AnrqFENshlpgEfh_u-V5OsPWL6p3-KdPN2XWIC7H2nsEk0-mg9nTFjsYGHjKrMD2RjX1Xtm4ps=</user>
  <password>gAAAAABn6p9MU8shpKVTmEv_BjWQqyTmTadIrUotYJ1OWCTEWxYg5mG6dL_kWpaed2dfpkfCcutLtnc89kUZ94Wk4J-ijjqsAg==</password>
  <ovc>10.1.40.15</ovc>
  <logfile>svtdemo.log</logfile>
  <timerange>30</timerange>
  <resolution>SECOND</resolution>
  <monitoringintervall>30</monitoringintervall>
  <logfile>svtdemo.log</logfile>
  <port>9091</port>
  <monitor>fcn</monitor>
  <cluster></cluster>
  <limit>500</limit>
  <offset>-1</offset>
</data>

svtconnector.key:
----
R3eEo3h55iSucYSRrpSIZzo4LV7ydOuxK0TlW638flc=


BinaryData
====

Events:  <none>


In [27]:
kubectl -n prometheus delete configmap/svtinfrastructure-xml

configmap "svtinfrastructure-xml" deleted


In [28]:
kubectl -n prometheus describe configmap/svtinfra-xml

Name:         svtinfra-xml
Namespace:    prometheus
Labels:       <none>
Annotations:  <none>

Data
====
svtconnector.key:
----
R3eEo3h55iSucYSRrpSIZzo4LV7ydOuxK0TlW638flc=

svtconnector.xml:
----
<data> 
  <username>SvtCollector@vsphere.local</username>
  <user>gAAAAABn6p9M4nlNVN27GKiijxeKDXmn_EWpyCvay2xH-AnrqFENshlpgEfh_u-V5OsPWL6p3-KdPN2XWIC7H2nsEk0-mg9nTFjsYGHjKrMD2RjX1Xtm4ps=</user>
  <password>gAAAAABn6p9MU8shpKVTmEv_BjWQqyTmTadIrUotYJ1OWCTEWxYg5mG6dL_kWpaed2dfpkfCcutLtnc89kUZ94Wk4J-ijjqsAg==</password>
  <ovc>10.0.44.33</ovc>
  <timerange>30</timerange>
  <resolution>SECOND</resolution>
  <monitoringintervall>30</monitoringintervall>
  <logfile>svtdemo.log</logfile>
  <port>9091</port>
  <monitor>fcn</monitor>
  <cluster></cluster>
  <limit>500</limit>
  <offset>-1</offset>
</data>


BinaryData
====

Events:  <none>


## 3. Create the SimpliVity Connector POD and Service with the Config Maps

### 3.1 Create the SimpliVity Connector POD for the CTC Demo environment

In [30]:
cat << 'EOF' | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: svtdemo
  namespace: prometheus
  labels:
    app: svtdemo
spec:
  selector:
    matchLabels:
      app: svtdemo
  template:
    metadata:
      labels:
        app: svtdemo
    spec:
      containers:
        - name: svtdemo
          image: tb1378/svtconk8s
          command: ["/usr/bin/python3"]
          args: ["/opt/svt/svtpromconnector.py"]
          volumeMounts:
            - name: svtconnectorxml
              mountPath: /opt/svt/data
      volumes:
        - name: svtconnectorxml
          configMap:
            name: svtdemo-xml   # the correct name of the configmap needs to be added here. 
EOF

deployment.apps/svtdemo created


In [31]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: svtdemo
  namespace: prometheus
#  labels: 
#     hpecp.hpe.com/hpecp-internal-gateway: "true"    
spec:
  selector:
    app: svtdemo
  ports:
    - port: 9091               # The Port of that the SimpliVity connector uses
      targetPort: 9091
      protocol: TCP
  type: NodePort               # expose the SimpliVity connector if you want/need to debug it. 
EOF

service/svtdemo unchanged


In [33]:
kubectl -n prometheus get deployments
kubectl -n prometheus get svc

NAME      READY   UP-TO-DATE   AVAILABLE   AGE
svtdemo   0/1     1            0           14s
NAME      TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
svtdemo   NodePort   10.152.183.21   <none>        9091:30863/TCP   28m


In [37]:
kubectl -n prometheus get pods

NAME                       READY   STATUS             RESTARTS      AGE
svtdemo-556f465646-mvgfv   0/1     CrashLoopBackOff   5 (15s ago)   3m33s


In [38]:
kubectl -n prometheus logs pod/svtdemo-556f465646-mvgfv

2025-03-31 14:18:25.050559: Path:    /opt/svt/ 

2025-03-31 14:18:25.050568: Keyfile: /opt/svt/data/svtconnector.key 

2025-03-31 14:18:25.050572: XMLfile: /opt/svt/data/svtconnector.xml 

2025-03-31 14:18:25.083836: Open a connection to the SimpliVity systems 

2025-03-31 14:18:25.084255: Open Connection to SimpliVity 

Traceback (most recent call last):
  File "/opt/svt/svtpromconnector.py", line 227, in <module>
    svt.Connect(svtuser,svtpassword)
  File "/opt/svt/SimpliVityClass.py", line 104, in Connect
    raise SvtError('SimpliVity.connect',response.status_code, response.json())
SimpliVityClass.SvtError: ('SimpliVity.connect', 404, {'error_type': 'NOT_FOUND', 'messages': [{'args': [], 'default_message': 'Not found.', 'id': 'com.vmware.vapi.rest.httpNotFound'}]})


In [39]:
kubectl -n prometheus delete deployment/svtdemo

deployment.apps "svtdemo" deleted


### 3.2 Create the SimpliVity Connector POD for the CTC infrastructure cluster

In [28]:
cat << 'EOF' | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: svtinfra
  namespace: svtprometheus
  labels:
    app: svtinfra
spec:
  selector:
    matchLabels:
      app: svtinfra
  template:
    metadata:
      labels:
        app: svtinfra
    spec:
      containers:
        - name: svtinfra
          image: tb1378/svtconk8s
          command: ["/usr/bin/python3"]
          args: ["/opt/svt/svtpromconnector.py"]
          volumeMounts:
            - name: svtconnectorxml
              mountPath: /opt/svt/data
      volumes:
        - name: svtconnectorxml
          configMap:
            name: svtinfrastructure-xml   # the correct name of the configmap needs to be added here. 
 
EOF

deployment.apps/svtinfra created


In [12]:
cat << 'EOF' | kubectl apply -f -
---
apiVersion: v1
kind: Service
metadata:
  name: svtinfra
  namespace: svtprometheus
#  labels: 
#     hpecp.hpe.com/hpecp-internal-gateway: "true"    
spec:
  selector:
    app: svtinfra
  ports:
    - port: 9091               # The Port of that the SimpliVity connector uses
      targetPort: 9091
      protocol: TCP
  type: NodePort               # expose the SimpliVity connector if you want/need to debug it.
EOF

service/svtinfra created


In [24]:
kubectl -n svtprometheus get pods,svc -o wide

NAME                                                  READY   STATUS    RESTARTS       AGE    IP            NODE                      NOMINATED NODE   READINESS GATES
pod/alletra6k-hpe-array-exporter-fc6b7cddc-kgfsx      1/1     Running   0              171d   10.244.3.19   suo04k8sr-w1.demo.local   <none>           <none>
pod/alletra9060-hpe-array-exporter-6b749784c-2d6nm    1/1     Running   0              171d   10.244.4.17   suo04k8sr-w2.demo.local   <none>           <none>
pod/alletramp16-hpe-array-exporter-86cd697857-sxl88   1/1     Running   0              171d   10.244.5.12   suo04k8sr-w3.demo.local   <none>           <none>
pod/alletramp8-hpe-array-exporter-659d946545-zpwqh    1/1     Running   0              171d   10.244.5.13   suo04k8sr-w3.demo.local   <none>           <none>
pod/dhci-hpe-array-exporter-6d7c968b4b-wg2ch          1/1     Running   0              171d   10.244.3.20   suo04k8sr-w1.demo.local   <none>           <none>
pod/dnsutils                               

In [26]:
kubectl -n svtprometheus delete deployment/svtinfra

deployment.apps "svtinfra" deleted


# Delete Deployments

In [6]:
kubectl -n svtprometheus get pods,svc,pv,deployment

NAME                                    TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE
service/alletra6k-hpe-array-exporter    NodePort    10.108.226.130   <none>        9090:32714/TCP   334d
service/dhci-hpe-array-exporter         ClusterIP   10.98.60.160     <none>        9090/TCP         557d
service/grafana-service                 NodePort    10.99.178.61     <none>        3000:31200/TCP   590d
service/ilo2                            NodePort    10.110.182.5     <none>        9091:31107/TCP   248d
service/ilo3                            NodePort    10.109.45.7      <none>        9091:30291/TCP   244d
service/ilo4                            NodePort    10.108.224.80    <none>        9091:32761/TCP   244d
service/ilo5                            NodePort    10.100.192.31    <none>        9091:31598/TCP   243d
service/iloredfish                      NodePort    10.101.190.222   <none>        9091:31949/TCP   334d
service/ilotest                         NodePort    10.9

In [21]:
kubectl -n svtprometheus delete deployment/svtinfra

deployment.apps "svtinfra" deleted


In [23]:
kubectl -n svtprometheus delete deployment/svtdemo

deployment.apps "svtdemo" deleted
